In [ ]:
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org laspy 

In [13]:
import laspy
import numpy as np
from osgeo import osr

In [9]:
fname = r"C:\Users\talbertc\OneDrive - DOI\Projects\MDWiz_lasExtension\DemoData\2023-05-05_Site7_Lidar_PointCloud.las"
# fname = "/mnt/c/Users/talbertc/OneDrive - DOI/Projects/MDWiz_lasExtension/DemoData/2023-05-05_Site7_Lidar_PointCloud.las"

In [10]:
with laspy.open(fname) as fh:
    print('Points from Header:', fh.header.point_count)
    las = fh.read()
    print(las)
    print('Points from data:', len(las.points))
    ground_pts = las.classification == 2
    bins, counts = np.unique(las.return_number[ground_pts], return_counts=True)
    print('Ground Point Return Number distribution:')
    for r,c in zip(bins,counts):
        print('    {}:{}'.format(r,c))

Points from Header: 2581316
<LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 2581316 points, 3 vlrs)>
Points from data: 2581316
Ground Point Return Number distribution:
    1:2571356


In [11]:
crs = fh.header.parse_crs()

In [12]:
type(crs)

pyproj.crs.crs.CRS

In [18]:

srs = osr.SpatialReference(wkt=crs.to_wkt())

In [15]:
fh.header.maxs

array([6.383090e+05, 4.229943e+06, 1.786718e+03])

In [16]:
fh.header.y_max

4229943.0

In [24]:
def _get_las_extent(fh):
    header = fh.header
    ulx = header.x_min
    lrx = header.x_max
    uly = header.y_max
    lry = header.y_min
    return ulx, lrx, lry, uly

min_x, max_x, min_y, max_y = _get_las_extent(fh)

In [19]:
target = osr.SpatialReference()
target.ImportFromEPSG(4326)
target.SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER)

In [26]:
min_y, max_y

(4229873.0, 4229943.0)

In [20]:
spatialRef = srs
spatialRef.ExportToProj4()
spatialRef.AutoIdentifyEPSG()
spref = spatialRef.GetAuthorityCode(None)

In [22]:
def transform_point(xy, from_srs, to_srs):
    """
    Transforms a point from one srs to another

    Parameters
    ----------
    x : float
    y : float
    from_srs : ogr projection
    to_srs : ogr projection

    Returns
    -------
    (x, y) : (float, float)
    """
    coord_xform = osr.CoordinateTransformation(from_srs, to_srs)
    y_round = round(xy[1], 8)
    x_round = round(xy[0], 8)

    results = coord_xform.TransformPoint(x_round, y_round)
    return results[0], results[1]

In [25]:
# Basically just run transform_point until no inf values return (weird bug with transform_point)
try_again = True
while try_again:
    west, south  = transform_point((min_x, min_y),from_srs=srs, to_srs=target)
    east, north  = transform_point((max_x, max_y),from_srs=srs, to_srs=target)
    if float("inf") not in [south, west, north, east] and float("-inf") not in [south, west, north, east]:
        try_again = False
west, east, south, north

(-109.42111471382677,
 -109.42027901584203,
 38.20620953873869,
 38.206829117899225)